# 🚀 SSHX en Google Colab con Persistencia de Datos

Este notebook te permite ejecutar un servidor SSHX en Google Colab con **persistencia de datos**.

## 📋 Características:
- ✅ Servidor SSHX con acceso remoto
- ✅ Persistencia de datos en Google Drive
- ✅ Restauración automática de archivos
- ✅ Aprovecha 96 cores y 344 GB RAM (VP5-E)
- ✅ Instalaciones automáticas guardadas

## 🎯 Uso:
1. Ejecuta las celdas en orden
2. Autoriza Google Drive cuando se solicite
3. Copia la URL de SSHX generada
4. Conéctate desde tu PC con `pruebaVPS.py`

---


## 📦 Paso 1: Montar Google Drive para Persistencia

In [ ]:
# Montar Google Drive
from google.colab import drive
import os
import shutil
from datetime import datetime

print("📂 Montando Google Drive...")
drive.mount('/content/drive', force_remount=True)

# Crear directorios de persistencia
DRIVE_BASE = '/content/drive/MyDrive/SSHX_Persistencia'
DRIVE_HOME = f'{DRIVE_BASE}/home'
DRIVE_DATA = f'{DRIVE_BASE}/data'
DRIVE_CONFIG = f'{DRIVE_BASE}/config'
DRIVE_LOGS = f'{DRIVE_BASE}/logs'

# Crear estructura de directorios
for directory in [DRIVE_BASE, DRIVE_HOME, DRIVE_DATA, DRIVE_CONFIG, DRIVE_LOGS]:
    os.makedirs(directory, exist_ok=True)
    print(f"✅ Directorio creado/verificado: {directory}")

# Registrar inicio de sesión
timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
with open(f'{DRIVE_LOGS}/session_log.txt', 'a') as f:
    f.write(f"\n{'='*60}\n")
    f.write(f"Sesión iniciada: {timestamp}\n")
    f.write(f"{'='*60}\n")

print("\n✅ Google Drive montado y directorios de persistencia creados")
print(f"📁 Ubicación: {DRIVE_BASE}")

## 🔧 Paso 2: Configurar Entorno con Persistencia

In [ ]:
import os
import shutil
import subprocess

print("🔧 Configurando entorno persistente...\n")

# 1. Crear directorio de trabajo local
WORK_DIR = '/content/workspace'
os.makedirs(WORK_DIR, exist_ok=True)
os.chdir(WORK_DIR)

# 2. Restaurar archivos de sesión anterior
print("📥 Restaurando datos de sesión anterior...")
if os.path.exists(DRIVE_HOME) and os.listdir(DRIVE_HOME):
    # Copiar archivos guardados al workspace
    for item in os.listdir(DRIVE_HOME):
        src = os.path.join(DRIVE_HOME, item)
        dst = os.path.join(WORK_DIR, item)
        if os.path.isdir(src):
            if os.path.exists(dst):
                shutil.rmtree(dst)
            shutil.copytree(src, dst)
        else:
            shutil.copy2(src, dst)
    print(f"✅ Restaurados {len(os.listdir(DRIVE_HOME))} elementos")
else:
    print("ℹ️  No hay datos de sesión anterior")

# 3. Crear script de backup automático
backup_script = f'''#!/bin/bash
# Script de backup automático
echo "💾 Iniciando backup automático..."

# Sincronizar workspace con Google Drive
rsync -av --delete {WORK_DIR}/ {DRIVE_HOME}/ 2>&1 | tee -a {DRIVE_LOGS}/backup.log

echo "✅ Backup completado: $(date)" | tee -a {DRIVE_LOGS}/backup.log
'''

with open('/content/auto_backup.sh', 'w') as f:
    f.write(backup_script)
os.chmod('/content/auto_backup.sh', 0o755)

print("✅ Script de backup creado")

# 4. Configurar variables de entorno
os.environ['PERSISTENT_HOME'] = DRIVE_HOME
os.environ['PERSISTENT_DATA'] = DRIVE_DATA
os.environ['WORK_DIR'] = WORK_DIR

print("\n✅ Entorno configurado")
print(f"📁 Directorio de trabajo: {WORK_DIR}")
print(f"💾 Backup en: {DRIVE_HOME}")

## 📦 Paso 3: Instalar Herramientas (con Cache)

In [ ]:
import subprocess
import os

print("📦 Instalando herramientas...\n")

# Archivo de marcador de instalación
installed_marker = f'{DRIVE_CONFIG}/tools_installed.txt'

def install_tools():
    """Instala herramientas necesarias"""
    tools = [
        ('neofetch', 'apt-get install -y neofetch'),
        ('htop', 'apt-get install -y htop'),
        ('tmux', 'apt-get install -y tmux'),
        ('vim', 'apt-get install -y vim'),
        ('git', 'apt-get install -y git'),
        ('curl', 'apt-get install -y curl'),
        ('wget', 'apt-get install -y wget'),
    ]

    # Actualizar repositorios
    print("🔄 Actualizando repositorios...")
    subprocess.run('apt-get update -qq', shell=True, check=True)

    # Instalar cada herramienta
    installed = []
    for tool_name, install_cmd in tools:
        print(f"⚙️  Instalando {tool_name}...")
        try:
            subprocess.run(install_cmd, shell=True, check=True, 
                         stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
            installed.append(tool_name)
            print(f"✅ {tool_name} instalado")
        except Exception as e:
            print(f"⚠️  Error instalando {tool_name}: {e}")

    # Guardar lista de herramientas instaladas
    with open(installed_marker, 'w') as f:
        f.write(f"Instalado: {datetime.now()}\n")
        f.write("\n".join(installed))

    return installed

# Verificar si ya se instalaron las herramientas
if os.path.exists(installed_marker):
    print("ℹ️  Herramientas ya instaladas anteriormente")
    with open(installed_marker, 'r') as f:
        print(f.read())
    
    # Reinstalar de todos modos (rápido con cache de apt)
    print("\n🔄 Reinstalando para asegurar disponibilidad...")
    installed = install_tools()
else:
    installed = install_tools()

print(f"\n✅ Total instalado: {len(installed)} herramientas")

# Mostrar información del sistema
print("\n" + "="*60)
print("📊 INFORMACIÓN DEL SISTEMA")
print("="*60)
subprocess.run('neofetch', shell=True)

## 🚀 Paso 4: Iniciar SSHX Server

In [ ]:
import subprocess
import threading
import time
import re
from datetime import datetime

print("🚀 Iniciando SSHX Server...\n")

# Descargar e instalar SSHX
print("📥 Instalando SSHX...")
result = subprocess.run(
    'curl -sSf https://sshx.io/get | sh',
    shell=True,
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print("✅ SSHX instalado correctamente")
else:
    print(f"⚠️  Resultado de instalación: {result.stderr}")

# Función para ejecutar SSHX y capturar la URL
def run_sshx():
    global sshx_url, sshx_process
    
    print("\n🔗 Iniciando servidor SSHX...")
    print("⏳ Esperando URL de conexión...\n")
    
    # Ejecutar SSHX
    process = subprocess.Popen(
        'curl -sSf https://sshx.io/get | sh -s run',
        shell=True,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1
    )
    
    sshx_process = process
    sshx_url = None
    
    # Capturar output
    for line in process.stdout:
        print(line.rstrip())
        
        # Buscar la URL en el output
        if 'https://sshx.io/s/' in line:
            match = re.search(r'https://sshx\.io/s/[^\s]+', line)
            if match:
                sshx_url = match.group(0)
                
                # Guardar URL en Drive
                with open(f'{DRIVE_CONFIG}/last_sshx_url.txt', 'w') as f:
                    f.write(f"URL: {sshx_url}\n")
                    f.write(f"Timestamp: {datetime.now()}\n")
                
                # Mostrar URL destacada
                print("\n" + "="*70)
                print("🎉 SERVIDOR SSHX ACTIVO")
                print("="*70)
                print(f"\n🔗 URL de Conexión:")
                print(f"\n   {sshx_url}\n")
                print("📋 Instrucciones:")
                print("   1. Copia la URL de arriba")
                print("   2. Ejecuta pruebaVPS.py en tu PC Windows 11")
                print("   3. Pega la URL cuando se solicite")
                print("   4. ¡Disfruta de 96 cores y 344 GB RAM!")
                print("\n💾 Backup automático activo")
                print("   Tus datos se guardarán en Google Drive")
                print("\n⚠️  IMPORTANTE: No cierres esta celda")
                print("   El servidor se mantendrá activo mientras esta celda esté ejecutándose")
                print("\n" + "="*70 + "\n")

# Iniciar backup automático en segundo plano
def auto_backup():
    """Realiza backups automáticos cada 5 minutos"""
    while True:
        time.sleep(300)  # 5 minutos
        try:
            subprocess.run('/content/auto_backup.sh', shell=True)
        except Exception as e:
            print(f"⚠️  Error en backup automático: {e}")

# Iniciar thread de backup
backup_thread = threading.Thread(target=auto_backup, daemon=True)
backup_thread.start()
print("✅ Backup automático activado (cada 5 minutos)\n")

# Ejecutar SSHX (esto bloqueará la celda)
try:
    run_sshx()
except KeyboardInterrupt:
    print("\n⚠️  Servidor SSHX detenido")
    print("💾 Ejecutando backup final...")
    subprocess.run('/content/auto_backup.sh', shell=True)
    print("✅ Backup completado")

## 💾 Paso 5: Backup Manual (Ejecutar antes de cerrar)

In [ ]:
import subprocess
import os
from datetime import datetime

print("💾 Ejecutando backup manual...\n")

# Ejecutar script de backup
result = subprocess.run(
    '/content/auto_backup.sh',
    shell=True,
    capture_output=True,
    text=True
)

print(result.stdout)

# Mostrar estadísticas
print("\n📊 Estadísticas de Backup:")
print("="*60)

if os.path.exists(DRIVE_HOME):
    # Contar archivos
    total_files = 0
    total_size = 0
    
    for root, dirs, files in os.walk(DRIVE_HOME):
        total_files += len(files)
        for file in files:
            try:
                total_size += os.path.getsize(os.path.join(root, file))
            except:
                pass
    
    print(f"📁 Total de archivos: {total_files}")
    print(f"💽 Tamaño total: {total_size / (1024*1024):.2f} MB")
    print(f"📅 Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"📍 Ubicación: {DRIVE_HOME}")
    
    # Registrar en log
    with open(f'{DRIVE_LOGS}/session_log.txt', 'a') as f:
        f.write(f"\nBackup manual: {datetime.now()}\n")
        f.write(f"Archivos: {total_files}, Tamaño: {total_size / (1024*1024):.2f} MB\n")
    
    print("\n✅ Backup completado exitosamente")
    print("\n💡 Tus datos están seguros en Google Drive")
    print("   Se restaurarán automáticamente en la próxima sesión")
else:
    print("⚠️  No se encontró el directorio de backup")

print("\n" + "="*60)

## 🔍 Paso 6: Ver Logs y Estado

In [ ]:
import os
from datetime import datetime

print("📊 ESTADO DEL SISTEMA")
print("="*70)

# 1. Información de recursos
print("\n💻 Recursos del Sistema:")
print("-" * 70)
import subprocess

# CPU
cpu_info = subprocess.run('nproc', shell=True, capture_output=True, text=True)
print(f"🔹 CPU Cores: {cpu_info.stdout.strip()}")

# RAM
mem_info = subprocess.run(
    "free -h | grep Mem | awk '{print $2}'",
    shell=True,
    capture_output=True,
    text=True
)
print(f"🔹 RAM Total: {mem_info.stdout.strip()}")

# Disco
disk_info = subprocess.run(
    "df -h / | tail -1 | awk '{print $2}'",
    shell=True,
    capture_output=True,
    text=True
)
print(f"🔹 Disco Total: {disk_info.stdout.strip()}")

# 2. Estado de SSHX
print("\n🔗 Estado de SSHX:")
print("-" * 70)

url_file = f'{DRIVE_CONFIG}/last_sshx_url.txt'
if os.path.exists(url_file):
    with open(url_file, 'r') as f:
        print(f.read())
else:
    print("⚠️  No hay URL de SSHX guardada")

# 3. Archivos en workspace
print("\n📁 Archivos en Workspace:")
print("-" * 70)

if os.path.exists(WORK_DIR):
    items = os.listdir(WORK_DIR)
    if items:
        for item in items[:10]:  # Mostrar solo primeros 10
            path = os.path.join(WORK_DIR, item)
            if os.path.isdir(path):
                print(f"📂 {item}/")
            else:
                size = os.path.getsize(path)
                print(f"📄 {item} ({size:,} bytes)")
        
        if len(items) > 10:
            print(f"... y {len(items) - 10} más")
    else:
        print("📭 Workspace vacío")

# 4. Últimos backups
print("\n💾 Historial de Backups:")
print("-" * 70)

log_file = f'{DRIVE_LOGS}/session_log.txt'
if os.path.exists(log_file):
    with open(log_file, 'r') as f:
        lines = f.readlines()
        # Mostrar últimas 15 líneas
        for line in lines[-15:]:
            print(line.rstrip())
else:
    print("ℹ️  No hay logs disponibles")

print("\n" + "="*70)
print(f"⏰ Fecha actual: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*70)

## 🧹 Paso 7: Limpieza (Opcional)

In [ ]:
import shutil
import os

print("🧹 LIMPIEZA DE DATOS")
print("="*70)
print("\n⚠️  ADVERTENCIA: Esta acción eliminará TODOS los datos guardados")
print("   en Google Drive para este proyecto.\n")

confirm = input("¿Estás seguro? Escribe 'ELIMINAR TODO' para confirmar: ")

if confirm == "ELIMINAR TODO":
    print("\n🗑️  Eliminando datos...")
    
    try:
        if os.path.exists(DRIVE_BASE):
            shutil.rmtree(DRIVE_BASE)
            print("✅ Datos eliminados de Google Drive")
        
        if os.path.exists(WORK_DIR):
            shutil.rmtree(WORK_DIR)
            print("✅ Workspace local limpiado")
        
        print("\n✅ Limpieza completada")
        print("   Puedes empezar de nuevo ejecutando las celdas desde el inicio")
    
    except Exception as e:
        print(f"❌ Error durante la limpieza: {e}")
else:
    print("\n❌ Limpieza cancelada")
    print("   Tus datos están seguros")

---

## 📚 Notas Importantes

### 💾 Persistencia de Datos
- Tus archivos se guardan automáticamente en Google Drive cada 5 minutos
- Ejecuta el **Paso 5** (Backup Manual) antes de cerrar la sesión
- Los datos se restauran automáticamente en la próxima sesión

### 🔗 Conexión SSHX
- La URL de SSHX cambia en cada sesión
- Copia la nueva URL cada vez que inicies el notebook
- Usa `pruebaVPS.py` desde Windows 11 para conectarte

### ⚡ Rendimiento
- Asegúrate de seleccionar **TPU v5e** en configuración de notebook
- Verifica la forma de máquina: **High-RAM**
- Esto te dará acceso a ~96 cores y ~344 GB RAM

### 🛡️ Seguridad
- Las sesiones SSHX son efímeras y seguras
- No compartas la URL de SSHX públicamente
- Los datos en Google Drive son privados de tu cuenta

### 🆘 Solución de Problemas

**Problema**: SSHX no se inicia
- Reinicia el runtime de Colab
- Ejecuta las celdas en orden

**Problema**: Datos no se restauran
- Verifica que Google Drive esté montado
- Revisa el **Paso 6** para ver el estado

**Problema**: Conexión lenta
- Verifica tu conexión a internet
- Cambia de región de Colab si es necesario

---

### 🎯 Próximos Pasos

1. **En Google Colab**: Ejecuta las celdas en orden
2. **En tu PC Windows 11**: 
   ```bash
   python pruebaVPS.py
   ```
3. **Conéctate**: Pega la URL de SSHX cuando se solicite
4. **¡Disfruta!**: Tienes acceso a un servidor Linux potente

---

**Generado con Claude Code** - https://claude.com/claude-code
